In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
import os
os.listdir()


['.git',
 '.ipynb_checkpoints',
 '.venv',
 'ANN_PRO.ipynb',
 'app.py',
 'Churn_Modelling.csv',
 'label_encoder_gender.pk1',
 'logs',
 'model.h5',
 'my_model.keras',
 'onehot_encoder_geo.pk1',
 'prediction.ipynb',
 'README.md',
 'requirements.txt',
 'runtime.txt',
 'scaler.pk1']

In [5]:
import pickle
from tensorflow.keras.models import load_model

# Load trained Keras model
model = load_model('model.h5')

# Load OneHotEncoder for Geography
with open('onehot_encoder_geo.pk1', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

# Load LabelEncoder for Gender
with open('label_encoder_gender.pk1', 'rb') as file:
    label_encoder_gender = pickle.load(file)

# Load StandardScaler
with open('scaler.pk1', 'rb') as file:
    scaler = pickle.load(file)


In [6]:
# --- Cleaned input_data ---
input_data = {
    'CreditScore': 600,
    'Gender': 'Female',
    'Geography': 'France',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:

# Wrap input as DataFrame with correct column name
geo_input = pd.DataFrame({'Geography': [input_data['Geography']]})

# Transform without warning
geo_encoded = onehot_encoder_geo.transform(geo_input).toarray()
geo_columns = onehot_encoder_geo.get_feature_names_out(['Geography'])

# Create DataFrame
geo_df = pd.DataFrame(geo_encoded, columns=geo_columns)
geo_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Female,France,40,3,60000,2,1,1,50000


In [9]:
## encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,0,France,40,3,60000,2,1,1,50000


In [21]:
# --- Build DataFrame in correct order ---
input_df = pd.DataFrame([[
    input_data['CreditScore'],
    gender_encoded,
    input_data['Age'],
    input_data['Tenure'],
    input_data['Balance'],
    input_data['NumOfProducts'],
    input_data['HasCrCard'],
    input_data['IsActiveMember'],
    input_data['EstimatedSalary'],
    *geo_encoded.toarray().flatten()

]], columns=[
    'CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
    'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
    *geo_columns
])


In [23]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [25]:
# --- Predict ---
prediction = model.predict(input_scaled)
print("Churn Probability:", prediction[0][0])
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Churn Probability: 0.033913616


array([[0.03391362]], dtype=float32)

In [27]:
pridiction_prob=prediction[0][0]

In [29]:
pridiction_prob

0.033913616

In [31]:
if prediction > 0.5:
    print('this customer is likely to churn.')
else:
    print('this customer is not likely to churn')

this customer is not likely to churn
